# Vehicle Detection and Tracking

It's gonna be awesome... Trust me... I know the best code in the world. I knowhow to make Vehicle detection great.

In [1]:
import os
import glob
#First locate vehicle images
basedir = 'vehicles/'
image_types = os.listdir(basedir)
cars = []
for imtypes in image_types:
    cars.extend(glob.glob(basedir+imtypes+'/*'))
print('Number of car images found:', len(cars))
with open('cars.txt','w') as f:
    for fn in cars:
        f.write(fn +'\n')
#Do the samething for non-vehicle images
basedir = 'non-vehicles/'
image_types = os.listdir(basedir)
notcars = []
for imtypes in image_types:
    notcars.extend(glob.glob(basedir+imtypes+'/*'))
print('Number of non-car images found:', len(notcars))
with open('notcars.txt','w') as f:
    for fn in notcars:
        f.write(fn +'\n')


Number of car images found: 8792
Number of non-car images found: 8968


In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
#Define a function to return HOG extraction and visualization
def get_hog_features(img, orient, pixel_per_cell, cell_per_block, vis=False, feature_vec=True):
    #Call with two outputs if vis=True
    if vis == True:
        features, hog_image = hog(img, orientations = orient,
                                 pixels_per_cell = (pix_per_cell, pix_per_cell),
                                 cells_per_block = (cell_per_block, cell_per_block), 
                                 transform_sqrt = False,
                                 visualise=vis, feature_vector = feature_vec)
        return features, hog_image
    else:
        features = hog(img, orientations = orient,
                                 pixels_per_cell = (pix_per_cell, pix_per_cell),
                                 cells_per_block = (cell_per_block, cell_per_block), 
                                 transform_sqrt = False,
                                 visualise=vis, feature_vector = feature_vec)
        return features


In [4]:
#Define a function to compute binned color features
#def bin_spatial(img, size=(32,32)):
    #Use cv2.resize().ravel() to create the feature vector
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

#Define a function to compare color histogram features
#Need to change bons_range if reading .png files with mpimg!
def color_hist(img, nbins=32): 
    # Compute the histogram of the color channels seperately
    channel1_hist = np.histogram(img[:,:,0], bins = nbins)
    channel2_hist = np.histogram(img[:,:,0], bins = nbins)
    channel3_hist = np.histogram(img[:,:,0], bins = nbins)
    #concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return hist_features

#define a function to extract features from a list of images
#Have this function call bin_spacial() and color_histogram()
def extract_features(imgs, color_space='RGB', spatial_size=(32,32),
                    hist_bins=32, orient=9,
                    pix_per_cell=8, cell_per_block=2, hog_channel=0,
                    spatial_feat=True, hist_feat=True, hog_feat=True):
    #Create a list to append feature vectors to
    features=[]
    #Iterate through the list of images
    for file in imgs:
        file_features = []
        #Read in each one by one
        #image = mpimg.imread(file)
        #apply color conversion if other than 'RGB'
        if color_space!= 'RGB':
            if color_space == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif color_space == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif color_space == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif color_space == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size = spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            #Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True: 
            #Call hog_get_features() with vis = False, feature_vec = True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel],
                                                        orient, pix_per_cell, cell_per_block,
                                                        vis=False, feature_vec=True))
                hog_featuers = np.ravel(hog_features)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],orient,
                                               pix_per_cell. cell_per_block, vis= False, feature_vec=True)
            #append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    #Return the list of feature vectors
    return features


In [5]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [7]:
# Read in cars and notcars
vehicles = glob.glob('vehicles/*/*.png')
non_vehicles = glob.glob('non-vehicles/*/*.png')

# sample_size = 8000
# vehicles = vehicles[0:sample_size]
# non_vehicles = non_vehicles[0:sample_size]

print("Loading", (np.asarray(cars).shape), "vehicle images...")
print("Loading", (np.asarray(notcars).shape), "non-vehicle images...")

cars = []
notcars = []
for image in non_vehicles:
    notcars.append(mpimg.imread(image))
for image in vehicles:
    cars.append(mpimg.imread(image))
### TODO: Tweak these parameters and see how the results change.
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        

# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)

# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

# Use a linear SVC 
svc = LinearSVC()

# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

# Check the prediction time for a single sample
t=time.time()

Loading (8792, 64, 64, 3) vehicle images...
Loading (8968, 64, 64, 3) non-vehicle images...


TypeError: src is not a numpy array, neither a scalar

In [ ]:
# Define a function that takes in an image, start and stop position in both x and y, window size (x and y dimentions), and overlap fraction (for both x and y)
def sliding_window(img, x_start_stop=[None,None], y_start_stop=[None,None],
                  xy_window=(64,64), xy_overlap=(0.5,0.5)):
    #If  and/or y start/stop positions are not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    #Compute the span of the region to be searched
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1-xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1-xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_windows = np.int(xspan/nx_pix_per_step) - 1
    ny_windows = np.int(yspan/ny_pix_per_step) - 1
    #Initialize a list to append window positions to
    window_list = []
    #Loop through dinding x and y windows positions
    #Note: you could vectorize this step, but in practice
    #you will be considering windows one by one with your
    #classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            #Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            #Append window position to the list
            window_list.append(((startx, starty),(endx, endy)))
    #Return the list of windows
    return window_list

#Desine a function to draw bounding boxes
def draw_bboxes(img, bboxes, color=(0,0,255), thick=6):
    #Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        #draw a rectangle given the bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    #Return the image copy with boxes drawn
    return imcopy

#Define a function to extract features from a single image window
def single_img_features(img, color_space='RGB', spatial_size=(32,32),
                    hist_bins=32, orient=9,
                    pix_per_cell=8, cell_per_block=2, hog_channel=0,
                    spatial_feat=True, hist_feat=True, hog_feat=True, vis=False):
    #Create a list to append feature vectors to
    img_features=[]
    #Iterate through the list of images
    if color_space!= 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(image)

    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size = spatial_size)
        print('spatial_features: ', spatial_features.shape)
        img_features.append(spatial_features)
    if hist_feat == True:
        #Apply color_hist()
        hist_features = color_hist(feature_image, nbins=hist_bins)
        print('hist_features: ', hist_features.shape)
        img_features.append(hist_features)
    if hog_feat == True: 
        #Call hog_get_features() with vis = False, feature_vec = True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                                    orient, pix_per_cell, cell_per_block,
                                                    vis=False, feature_vec=True))
            hog_featuers = np.concatenate(hog_features)
        else:
            if vis == True:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],orient,
                                           pix_per_cell. cell_per_block, vis= True, feature_vec=True)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel],orient,
                                           pix_per_cell. cell_per_block, vis= False, feature_vec=True)
        print('hog_features: ', np.asarray(hog_features).shape)
        #append the new feature vector to the features list
        img_features.append(np.asarray(hog_features))
        print('img_features:',np.asarray(img_features).shape)
    #Return the list of feature vectors
    
    if vis == True: 
        #print(np.concatenate(img_features))
        #return np.concatenate(img_features), hog_image
        return img_features, hog_image
    else:
        return np.concatenate(img_features)
#Define a function you will pass an image
#and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB',
                  spatial_size=(32,32), hist_bins=32,
                  hist_range=(0,256), orient=9,
                  pix_per_cell=8, cell_per_block=2,
                  hog_channel=0, spatial_feat=True,
                  hist_feat=True, hog_feat=True):
    #Create an empty list to receive positive detection windows
    on_windows=[]
    #Iterate over all windows in the list
    for window in windows:
        #Extract the test window from original image
        test_img = cv2.resize(img[window[0][1]:windo[1][1], window[0][0]:window[1][0]], (64,64))
        #Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space=color_space,
                                      spatial_size=spatial_size,hist_bins=hist_bins,
                                      orient=orient, pix_per_cell=pix_per_cell,
                                      cell_per_block=cell_per_block,
                                      hog_channel=hog_channel, spatial_feat=spatial_feat,
                                      hist_feat=hist_feat, hog_feat=hog_feat)
        #Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1,-1))
        #Predict using your classifier
        prediction = clf.predict(test_features)
        #If positive (prediction ==1), then save the window
        if prediction ==1:
            on_windows.append(window)
    #Return windows for positive detection
    return on_windows

def visualize(fig, rows, cols, imgs, titles):
    for i, img in enumerate(image):
        plt.subplot(rows, cols, i+1)
        plt.title(i+1)
        img_dims = len(img.shape)
        if img_dims < 3:
            plt.imshow(img, cmap='hot')
            plt.title(titles[i])
        else:
            plt.imshow(img)
            plt.title(titles[i])

In [ ]:
%matplotlib inline

#Choose random car/notcar indices
print(len(cars))
print(len(notcars))
car_ind = np.random.randint(0,len(cars)-1)
notcar_ind = np.random.randint(0,len(notcars)-1)

#Read in car/noncar images
#print(cars[car_ind])
#print(notcars[notcar_ind])
car_image = cars[car_ind]
notcar_image = notcars[notcar_ind]
plt.imshow(car_image)
plt.show()
plt.imshow(notcar_image)
plt.show()
#Define feature parameters
color_spave = 'RGB' #Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 6
pix_per_cell= 8
cell_per_block = 2
hog_channels = 0 # Can be 0, 1, 2, "ALL"
spatial_size = (16,16)
hist_bins = 16
spatial_feat = True
hist_feat = True
hog_feat = True

car_features, car_hog_image = single_img_features(car_image, color_space=color_space, spatial_size=spatial_size,
                                                 hist_bins=hist_bins, orient=orient,
                                                 pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                                                 spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat, vis=True)
notcar_features, car_hog_image = single_img_features(notcar_image, color_space=color_space, spatial_size=spatial_size,
                                                 hist_bins=hist_bins, orient=orient,
                                                 pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                                                 spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat, vis=True)
images = [car_image, car_hog_image, notcar_image, notcar_hog_image]
titles = ['car image', 'car HOG image', 'notcar image', 'notcar HOG image']
fig = plt.figure(figsize=(12,3))
visualize(fig, 1,4,images, titles)
